In [ ]:
import scanpy as sc
import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys
import getopt

In [ ]:
#  plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rc('font', size=14)
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
study = "Ramachandran"
data_dir = "/opt/datastore/aiakovliev/liver"
input_dir = os.path.join(data_dir, study)
file_list = os.path.join(input_dir, "metadata.txt")
sample_dirs = set(pd.read_csv(file_list)["sample.dir"].tolist())
print(sample_dirs)

In [ ]:
this_dir = sample_dirs[0]
print(this_dir)
# load count matrix in mtx format
counts_matrix = sc.read_10x_mtx(
    this_dir,  # the directory with the `.mtx` file
    var_names='gene_symbols',   # use gene symbols for the variable names (genes)
    cache=True                 # cache the data
)
# alternatively laod count matrix in h5 format (if available)
if False: 
    counts_matrix = sc.read_10x_h5(this_dir)
print(counts_matrix.__class__.__name__)
# counts_matrix = sc.read_10x_h5(os.path.join(this_dir, "matrix.mtx"))
#counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx').T.tocsc()
#genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
#print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
#print('Number of genes in gene list: {}'.format(len(genes)))

In [ ]:
# doublet rates in 1000 cells normally ~0.8% per 1k cells
edr = 0.008
expected_doublet_rate = float(edr)*counts_matrix.shape[0]/1000
print('EDR: {}'.format(expected_doublet_rate))
scrub = scr.Scrublet(counts_matrix.X, expected_doublet_rate = float(expected_doublet_rate))
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2,
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=60, 
                                                          n_prin_comps=30,
                                                          log_transform=True,
                                                          mean_center=True,
                                                          normalize_variance=True,
                                                          synthetic_doublet_umi_subsampling = 1)
# after examining the bimodal distribution by eye, insert the threshold here
scrub.call_doublets(threshold=0.20)
scrub.plot_histogram()

outf1 = this_dir + "/scrublet_EDR" + str(edr) + "_DoubletScores.csv"
outf2 = this_dir + "/scrublet_EDR" + str(edr) + "_PredictedDoublets.csv"
np.savetxt(outf1, doublet_scores, delimiter=',')
np.savetxt(outf2, predicted_doublets, delimiter=',')
print("Saved " + outf1)
print("Saved " + outf2)

In [ ]:
for s in sample_dirs:
    outf1 = s + "/scrublet_EDR" + str(edr) + "_DoubletScores.csv"
    print(outf1)

In [ ]:
%matplotlib inline

for s in sample_dirs:
    
    # load count matrix
    if False:
        counts_matrix=sc.read_10x_h5(s)

    counts_matrix=sc.read_10x_mtx(
        s,
        var_names='gene_symbols',
        cache=True
    )
    
    # doublet rates in 1000 cells normally ~0.8% per 1k cells
    edr = 0.008
    expected_doublet_rate = float(edr)*counts_matrix.shape[0]/1000
    print('EDR: {}'.format(expected_doublet_rate))
    
    scrub = scr.Scrublet(counts_matrix.X, expected_doublet_rate = float(expected_doublet_rate))
    
    doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2,
                                                      min_cells=3, 
                                                      min_gene_variability_pctl=60, 
                                                      n_prin_comps=30,
                                                      log_transform=True,
                                                      mean_center=True,
                                                      normalize_variance=True,
                                                      synthetic_doublet_umi_subsampling = 1)
    
    # after examining the bimodal distribution by eye, insert the threshold here
    scrub.call_doublets(threshold=0.20)
    scrub.plot_histogram()

    outf1 = s + "/scrublet_EDR" + str(edr) + "_DoubletScores.csv"
    outf2 = s + "/scrublet_EDR" + str(edr) + "_PredictedDoublets.csv"
    np.savetxt(outf1, doublet_scores, delimiter=',')
    np.savetxt(outf2, predicted_doublets, delimiter=',')
    print("Saved " + outf1)
    print("Saved " + outf2)


## Check 2 of the samples that look different

In [ ]:
# load count matrix
counts_matrix = sc.read_10x_mtx(
    this_dir,  # the directory with the `.mtx` file
    var_names='gene_symbols',   # use gene symbols for the variable names (genes)
    cache=True                 # cache the data
)
# alternatively laod count matrix in h5 format (if available)
if False: 
    counts_matrix = sc.read_10x_h5(this_dir)
    
# doublet rates in 1000 cells normally ~0.8% per 1k cells
edr = 0.008
expected_doublet_rate = float(edr)*counts_matrix.shape[0]/1000
print('EDR: {}'.format(expected_doublet_rate))
    
scrub = scr.Scrublet(counts_matrix.X, expected_doublet_rate = float(expected_doublet_rate))
    
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2,
                                                          min_cells=3,
                                                          min_gene_variability_pctl=60, 
                                                          n_prin_comps=30,
                                                          log_transform=True,
                                                          mean_center=True,
                                                          normalize_variance=True,
                                                          synthetic_doublet_umi_subsampling = 1)
    
# after examining the bimodal distribution by eye, insert the threshold here
scrub.call_doublets(threshold=0.2)
scrub.plot_histogram()